## 📚 Imported Libraries

In [1]:
import pandas as pd
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import matplotlib.pyplot as plt
import os
import logging
import requests
import bs4
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
# Launch the browser
driver = webdriver.Chrome()
driver.get("https://en.wikipedia.org/wiki/Key_events_of_the_20th_century")

In [3]:
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CLASS_NAME, "mw-heading4"))
)


<selenium.webdriver.remote.webelement.WebElement (session="2088b1ee5025e46d94a9483d3083ec19", element="f.C4878FB9701FDED1FCB2E33687AE0548.d.6DB2A5C971AF56B87E39F89EDFF7293C.e.114")>

In [4]:
# Find all year/event headings
year_elems = driver.find_elements(By.CLASS_NAME, "mw-heading4")

# Preview the first few
for i in range(5):  # Adjust as needed
    print(year_elems[i].text)

"The war to end all wars": World War I (1914–1918)[edit]
Russian Revolution and communism[edit]
Economic depression[edit]
The rise of dictatorship[edit]
The war in Europe[edit]


In [5]:
# Find the main article container
article = driver.find_element(By.ID, "mw-content-text")

# Then extract only paragraph tags inside the main content
paragraphs = article.find_elements(By.TAG_NAME, "p")

# Combine all paragraph text
text = ""
for p in paragraphs:
    text += p.text + "\n"

# Save it to a clean .txt file
with open("20th_century_events.txt", "w", encoding="utf-8") as file:
    file.write(text)

print("Scraping complete and file saved.")



Scraping complete and file saved.


In [9]:
import re

years = []
for p in paragraphs:
    found_years = re.findall(r"\b(19|20)\d{2}\b", p.text)
    years.extend(found_years)

print(set(years))  # Unique years


{'20', '19'}


In [10]:
links = driver.find_elements(By.TAG_NAME, "a")
for link in links[:5]:
    print(link.get_attribute("href"))


https://en.wikipedia.org/wiki/Key_events_of_the_20th_century#bodyContent
https://en.wikipedia.org/wiki/Main_Page
https://en.wikipedia.org/wiki/Wikipedia:Contents
https://en.wikipedia.org/wiki/Portal:Current_events
https://en.wikipedia.org/wiki/Special:Random


In [11]:
images = driver.find_elements(By.TAG_NAME, "img")
for img in images[:5]:
    print(img.get_attribute("src"))


https://en.wikipedia.org/static/images/icons/wikipedia.png
https://en.wikipedia.org/static/images/mobile/copyright/wikipedia-wordmark-en.svg
https://en.wikipedia.org/static/images/mobile/copyright/wikipedia-tagline-en.svg
https://upload.wikimedia.org/wikipedia/commons/thumb/8/85/Ferdinand_Behr_arrested_in_Sarajevo_1914.jpg/250px-Ferdinand_Behr_arrested_in_Sarajevo_1914.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Dissolution_of_Austria-Hungary.png/250px-Dissolution_of_Austria-Hungary.png


In [12]:
from IPython.display import Image, display

# Show top 5 images that are not icons or logos (optional filter)
for img in images:
    src = img.get_attribute("src")
    if src and src.endswith((".jpg", ".jpeg", ".png")) and "commons" in src:
        display(Image(url=src))


In [14]:
# Combine scraped data
output = []

# Headings (h4)
headings = driver.find_elements(By.CLASS_NAME, "mw-heading4")
output.append("== Headings ==")
for h in headings:
    output.append(h.text)

# Paragraphs (p)
paragraphs = driver.find_elements(By.TAG_NAME, "p")
output.append("\n== Paragraphs ==")
for p in paragraphs[:5]:  # Only taking top 5 for now
    output.append(p.text)


# Image URLs
images = driver.find_elements(By.TAG_NAME, "img")
output.append("\n== Image Links ==")
for img in images[:5]:  # Only top 5
    src = img.get_attribute("src")
    if src:
        output.append(src)

# Save all to a text file
with open("20th_century_scrape.txt", "w", encoding="utf-8") as f:
    for line in output:
        f.write(line + "\n")

print("✅ File saved as '20th_century_keyevents.txt'")


✅ File saved as '20th_century_keyevents.txt'


In [4]:
import re
from collections import defaultdict

# Load the article content
with open("20th_century_events.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Create dictionary to group events by year
years_dict = defaultdict(list)

# Regex pattern for 4-digit years
year_pattern = r'\b(1[0-9]{3}|20[0-9]{2})\b'

# Split the text into sentences
sentences = re.split(r'(?<=\.|\?|!)\s+', text)

# Loop through sentences and collect years
for sentence in sentences:
    years_found = re.findall(year_pattern, sentence)
    for year in years_found:
        years_dict[year].append(sentence.strip())

# Display grouped events
for year in sorted(years_dict.keys()):
    print(f"\n🗓 {year}")
    for event in years_dict[year]:
        print(f" - {event}")



🗓 1910
 - The Korean Peninsula was a Japanese colony between 1910 and 1945, when Soviet and American troops invaded and divided it along the 38th parallel.[198] A communist government controlled the territory north of the border and a capitalist one controlled the South, with both authorities considering the other one illegitimate and claiming sovereignty over the entire peninsula.[199] North Korea's invasion of South Korea on 25 June 1950 led to United Nations intervention.[200] General Douglas MacArthur led troops from the United States, Canada, Australia, Great Britain, and other countries in repulsing the Northern invasion.

🗓 1914
 - 1914 saw the completion of the Panama Canal.
 - From 1914 to 1918, the First World War, and its aftermath, caused major changes in the power balance of the world, destroying or transforming some of the most powerful empires.
 - The First World War (or simply WWI), termed "The Great War" by contemporaries, started in July 1914 and ended in November 19

In [1]:
import re

# Let's say this is your raw scraped text
with open('20th_century_events.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

# Remove citations like [144], [26], etc.
clean_text = re.sub(r'\[\d+\]', '', raw_text)

# Remove non-ASCII characters (like corrupted characters or special symbols)
clean_text = re.sub(r'[^\x00-\x7F]+', '', clean_text)

# Optional: remove excessive whitespace
clean_text = re.sub(r'\s+', ' ', clean_text).strip()

# Save to a new file (optional)
with open('cleaned_20th_century_events.txt', 'w', encoding='utf-8') as f:
    f.write(clean_text)

print("🧽 Cleaned text saved successfully!")


🧽 Cleaned text saved successfully!


In [2]:
# Load cleaned text
with open("cleaned_20th_century_events.txt", "r") as file:
    cleaned_text = file.read()

# Optional: use regex to split based on sentence enders
import re
sentences = re.split(r'(?<=[.!?])\s+', cleaned_text.strip())

# Save it back with line breaks
with open("formatted_cleaned_20th_century_events.txt", "w") as file:
    for sentence in sentences:
        file.write(sentence.strip() + '\n')
